In [ ]:
import yaml
import add_packages
from pprint import pprint
import os, re
import pandas as pd
# import tqdm
from tqdm.auto import tqdm

from toolkit.langchain import (
	document_loaders, text_splitters, text_embedding_models, stores, 
	prompts, utils, output_parsers, agents, documents, models,
	runnables, tools, chains
)

from toolkit import sql

PATH_DATA = f"{add_packages.APP_PATH}/data/..."
FILE_CFG = "....yaml"
tqdm.pandas(desc="Processing")

with open(f"{add_packages.APP_PATH}/my_configs/{FILE_CFG}", 'r') as file:
    configs = yaml.safe_load(file)

In [ ]:
llm = models.chat_openai
my_sql_db = sql.MySQLDatabase()


# Data

## txt - FAQ

### File 1

In [ ]:
path_txt = f"{PATH_DATA}/faq.txt"

In [ ]:
loader_txt = document_loaders.TextLoader(path_txt)
doc_txt = loader_txt.load()

text_splitter = text_splitters.RecursiveCharacterTextSplitter(
	# chunk_size=500, chunk_overlap=100,
	separators=["##"], chunk_size=150, chunk_overlap=0,
)
docs_txt = text_splitter.split_documents(doc_txt)
docs_txt = docs_txt[1:]

metadatas = {
	"data": "frequently asked questions"
}
utils.remove_metadata(docs_txt, "source")
utils.update_metadata(docs_txt, metadatas)

### File 2

In [ ]:
path_txt = f"{PATH_DATA}/faq.txt"

In [ ]:
loader_txt = document_loaders.TextLoader(path_txt)
doc_txt = loader_txt.load()

text_splitter = text_splitters.RecursiveCharacterTextSplitter(
	# chunk_size=500, chunk_overlap=100,
	separators=["##"], chunk_size=150, chunk_overlap=0,
)
docs_txt = text_splitter.split_documents(doc_txt)
docs_txt = docs_txt[1:]

metadatas = {
	"data": "frequently asked questions"
}
utils.remove_metadata(docs_txt, "source")
utils.update_metadata(docs_txt, metadatas)

## table

### File 1

In [ ]:
file_xlsx = "..."
path_xlsx = f"{PATH_DATA}/{file_xlsx}"
path_xlsx_processed = f"{PATH_DATA}/{file_xlsx.split('.')[0]}1.xlsx"

In [ ]:
df = pd.read_excel(
	path_xlsx, 
 	# delimiter=";"
)

df.head()

# Prompting to get new col names


#### Process

In [ ]:
model = models.chat_openai

template1 = """\
...
{text}"""

template2 = """\
...
{text}
"""

prompt_template1 = prompts.PromptTemplate.from_template(template1)
prompt_template2 = prompts.PromptTemplate.from_template(template2)

chain1 = prompt_template1 | model | output_parsers.StrOutputParser()
chain2 = prompt_template2 | model | output_parsers.StrOutputParser()

chain = runnables.RunnablePassthrough.assign(
  text=chain1
).assign(
  text=chain2
)

def process_xlsx_col(text: str) -> str:
  result = chain.invoke({"text": text})['text']
  return result

def capitalize_first_letter(s):
	return ' '.join([word.capitalize() for word in s.split()])

query = '...'
result = process_xlsx_col(query)

pprint(result)

In [ ]:
df = pd.read_excel(path_xlsx)

col_to_process = "..."

df[col_to_process] = df[col_to_process].progress_apply(process_xlsx_col)

df.to_excel(f"{path_xlsx_processed}", index=False)


In [ ]:
path_xlsx = path_xlsx_processed

#### Load

In [ ]:
xlsx_cols = ...

loader_xlsx = document_loaders.UnstructuredExcelLoader(
	path_xlsx,
	mode="elements",
)
docs_xlsx = loader_xlsx.load()

metadatas = {
	"data": "..."
}

utils.remove_metadata(docs_xlsx, "source")
utils.remove_metadata(docs_xlsx, "row")
utils.update_metadata(docs_xlsx, metadatas)

#### Load to sql

In [ ]:
my_table_schema = [
	"course_id SERIAL",
	"course_name VARCHAR(255) NOT NULL UNIQUE",
	"course_category VARCHAR(255) NOT NULL",
	"instructor_name VARCHAR(100) NOT NULL",
	"course_link VARCHAR(2048) NOT NULL UNIQUE",
	"course_description TEXT NOT NULL",
	"PRIMARY KEY (course_id)",
]
my_table = sql.MySQLTable(
	name="...", 
	schema=my_table_schema,
	db=my_sql_db,
)
my_table.create()

db = stores.SQLDatabase.from_uri(my_sql_db.get_uri())
llm = models.chat_openai

embeddings = text_embedding_models.OpenAIEmbeddings()
vectorstore = stores.faiss.FAISS

table_cols = [col_description.split(" ")[0] for col_description in my_table_schema][1:-1]


In [ ]:
df = pd.read_excel(path_xlsx)
df.columns = table_cols

my_table.insert_from_dataframe(df)

cols = [...]
proper_nouns = [value for col in cols for value in my_table.get_discrete_values_col(col)]


In [ ]:
questions = ...
examples_questions_to_sql = ...

### File 2

In [ ]:
file_xlsx = "..."
path_xlsx = f"{PATH_DATA}/{file_xlsx}"
path_xlsx_processed = f"{PATH_DATA}/{file_xlsx.split('.')[0]}1.xlsx"

In [ ]:
df = pd.read_excel(
	path_xlsx, 
 	# delimiter=";"
)

df.head()


#### Process

In [ ]:
model = models.chat_openai

template1 = """\
...
{text}"""

template2 = """\
...
{text}
"""

prompt_template1 = prompts.PromptTemplate.from_template(template1)
prompt_template2 = prompts.PromptTemplate.from_template(template2)

chain1 = prompt_template1 | model | output_parsers.StrOutputParser()
chain2 = prompt_template2 | model | output_parsers.StrOutputParser()

chain = runnables.RunnablePassthrough.assign(
  text=chain1
).assign(
  text=chain2
)

def process_xlsx_col(text: str) -> str:
  result = chain.invoke({"text": text})['text']
  return result

query = '...'
result = process_xlsx_col(query)

pprint(result)

In [ ]:
df = pd.read_excel(path_xlsx)

col_to_process = "..."

df[col_to_process] = df[col_to_process].progress_apply(process_xlsx_col)

df.to_excel(f"{path_xlsx_processed}", index=False)


In [ ]:
path_xlsx = path_xlsx_processed

#### Load

In [ ]:
xlsx_cols = ...

loader_xlsx = document_loaders.UnstructuredExcelLoader(
	path_xlsx,
	mode="elements",
)
docs_xlsx = loader_xlsx.load()

metadatas = {
	"data": "..."
}

utils.remove_metadata(docs_xlsx, "source")
utils.remove_metadata(docs_xlsx, "row")
utils.update_metadata(docs_xlsx, metadatas)

#### Load to sql

In [ ]:
my_table_schema = [
	"course_id SERIAL",
	"course_name VARCHAR(255) NOT NULL UNIQUE",
	"course_category VARCHAR(255) NOT NULL",
	"instructor_name VARCHAR(100) NOT NULL",
	"course_link VARCHAR(2048) NOT NULL UNIQUE",
	"course_description TEXT NOT NULL",
	"PRIMARY KEY (course_id)",
]
my_table = sql.MySQLTable(
	name="...", 
	schema=my_table_schema,
	db=my_sql_db,
)
my_table.create()

db = stores.SQLDatabase.from_uri(my_sql_db.get_uri())
llm = models.chat_openai

embeddings = text_embedding_models.OpenAIEmbeddings()
vectorstore = stores.faiss.FAISS

table_cols = [col_description.split(" ")[0] for col_description in my_table_schema][1:-1]


In [ ]:
df = pd.read_excel(path_xlsx)
df.columns = table_cols

my_table.insert_from_dataframe(df)

cols = [...]
proper_nouns = [value for col in cols for value in my_table.get_discrete_values_col(col)]


# Vector store 

Note:
- `tiktoken` >= 0.6.0

## txt

### File 1

In [ ]:
qdrant_txt = stores.QdrantWrapper(
  qdrant_host=os.getenv("QDRANT_HOST"),
  qdrant_api_key=os.getenv("QDRANT_API_KEY"),
  configs=configs,
  **configs["vector_db"]["qdrant"]["..."]
)

In [ ]:
qdrant_txt.add_documents(docs_txt)

### File 2

In [ ]:
qdrant_txt = stores.QdrantWrapper(
  qdrant_host=os.getenv("QDRANT_HOST"),
  qdrant_api_key=os.getenv("QDRANT_API_KEY"),
  configs=configs,
  **configs["vector_db"]["qdrant"]["..."]
)

In [ ]:
# qdrant_txt.add_documents(docs_txt)

## table

### File 1

In [ ]:
qdrant_xlsx = stores.QdrantWrapper(
	qdrant_host=os.getenv("QDRANT_HOST"),
	qdrant_api_key=os.getenv("QDRANT_API_KEY"),
	configs=configs,
	**configs["vector_db"]["qdrant"]["..."]
)

In [ ]:
qdrant_xlsx.add_documents(docs_xlsx)

### File 2

In [ ]:
qdrant_xlsx = stores.QdrantWrapper(
	qdrant_host=os.getenv("QDRANT_HOST"),
	qdrant_api_key=os.getenv("QDRANT_API_KEY"),
	configs=configs,
	**configs["vector_db"]["qdrant"]["..."]
)

In [ ]:
qdrant_xlsx.add_documents(docs_xlsx)

# Test

In [ ]:
llm = models.chat_openai

tools = [
	tools.TavilySearchResults(max_results=3),
	qdrant_txt.retriever_tool,
	qdrant_xlsx.retriever_tool,
]

system_message_custom = configs["prompts"]["system_message_vtc"]
prompt = prompts.create_prompt_tool_calling_agent(system_message_custom)

agent = agents.MyStatelessAgent(
	llm=llm,
	tools=tools,
	prompt=prompt,
	agent_type=configs["agents"]["type"],
	agent_verbose=False,
)

In [ ]:
questions = [

]


In [ ]:
input_message = questions[1]
# await agent.stream_conversable_agent(questions[2])
result = agent.invoke_agent(input_message)
# await agent.stream_agent(input_message)
pprint(result)